# Backtes the three PTS Methods

In [1]:
import pandas as pd

from Methods.DmHelpers import *
from Methods.CointHelpers import *
from Methods.CopulasHelpers import *

from Helpers.DataHelpers import *
from Helpers.PlotHelpers import *
from Helpers.ModuleHelpers import *
from Helpers.BacktestHelpers import *

In [2]:
# Donwnload data
tickers = pd.read_excel('Data/SandP500list.xlsx')
tickers = tickers[tickers['sector'] == 'Financials']
names = tickers['symbol'].tolist()

START_DATE = pd.to_datetime('2020-01-01') # Every year takes approximately 30 seconds for coint and dm
END_DATE = pd.to_datetime('2024-04-01') # test 6 months yyy

data = download_data(names, START_DATE, END_DATE)

[*********************100%***********************]  65 of 65 completed


6 Failed downloads:
['BRK.B', 'PBCT', 'WLTW', 'FRC', 'RE']: Exception('%ticker%: No timezone found, symbol may be delisted')
['SIVB']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


In [3]:
# Run rolling window overlapping backtest

start_date, end_date = START_DATE, START_DATE + pd.DateOffset(months=18)

all_backtest_results = run_rolling_window_backtest(data, start_date, end_date, END_DATE=END_DATE, dm_threshold=2, coint_threshold=1, copula_threshold=0.8):

dm_backtest_results, coint_backtest_results, copula_backtest_results = all_backtest_results

Training from 2020-01-01 00:00:00 to 2021-01-01 00:00:00 and testing from 2021-01-01 00:00:00 to 2021-07-01 00:00:00
Backtest Period: 1
Start Date: 2020-01-01 End Date: 2021-07-01
-------------------------------------------
Training from 2020-02-01 00:00:00 to 2021-02-01 00:00:00 and testing from 2021-02-01 00:00:00 to 2021-08-01 00:00:00
Backtest Period: 2
Start Date: 2020-02-01 End Date: 2021-08-01
-------------------------------------------
Training from 2020-03-01 00:00:00 to 2021-03-01 00:00:00 and testing from 2021-03-01 00:00:00 to 2021-09-01 00:00:00
Backtest Period: 3
Start Date: 2020-03-01 End Date: 2021-09-01
-------------------------------------------
Training from 2020-04-01 00:00:00 to 2021-04-01 00:00:00 and testing from 2021-04-01 00:00:00 to 2021-10-01 00:00:00
Backtest Period: 4
Start Date: 2020-04-01 End Date: 2021-10-01
-------------------------------------------
Training from 2020-05-01 00:00:00 to 2021-05-01 00:00:00 and testing from 2021-05-01 00:00:00 to 2021-11

/Users/lasse/Desktop/EBSUni/Msc_Semester_1/Empirical Finance/PairsTradingAll/Pairs-Trading-Strategy/Code/Methods/CopulasHelpers.py:211: RuntimeWarning: invalid value encountered in divide
  conditional_cdf = t.cdf(numerator / denominator, df=nu + 1)


Backtest Period: 15
Start Date: 2021-03-01 End Date: 2022-09-01
-------------------------------------------
Training from 2021-04-01 00:00:00 to 2022-04-01 00:00:00 and testing from 2022-04-01 00:00:00 to 2022-10-01 00:00:00
Backtest Period: 16
Start Date: 2021-04-01 End Date: 2022-10-01
-------------------------------------------
Training from 2021-05-01 00:00:00 to 2022-05-01 00:00:00 and testing from 2022-05-01 00:00:00 to 2022-11-01 00:00:00
Backtest Period: 17
Start Date: 2021-05-01 End Date: 2022-11-01
-------------------------------------------
Training from 2021-06-01 00:00:00 to 2022-06-01 00:00:00 and testing from 2022-06-01 00:00:00 to 2022-12-01 00:00:00
Backtest Period: 18
Start Date: 2021-06-01 End Date: 2022-12-01
-------------------------------------------
Training from 2021-07-01 00:00:00 to 2022-07-01 00:00:00 and testing from 2022-07-01 00:00:00 to 2023-01-01 00:00:00
Backtest Period: 19
Start Date: 2021-07-01 End Date: 2023-01-01
------------------------------------

In [18]:
# get cumulative returns for each method
dm_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(dm_backtest_results, data))
coint_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(coint_backtest_results, data)).fillna(0)
coint_average_returns_tcosts = coint_average_returns_tcosts.loc[pd.to_datetime('2021-01-01'):]
copula_average_returns_tcosts = average_daily_returns(get_daily_returns_backtest_transaction_costs(copula_backtest_results, data))

dm_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(dm_backtest_results, data))
coint_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(coint_backtest_results, data)).fillna(0)
coint_average_returns_no_tcosts = coint_average_returns_no_tcosts.loc[pd.to_datetime('2021-01-01'):]
copula_average_returns_no_tcosts = average_daily_returns(get_daily_returns_backtest(copula_backtest_results, data))

# Combine all dfs and save to excel
all_dfs = [dm_average_returns_tcosts, coint_average_returns_tcosts, copula_average_returns_tcosts, dm_average_returns_no_tcosts, coint_average_returns_no_tcosts, copula_average_returns_no_tcosts]
all_dfs = pd.concat(all_dfs, axis=1)
all_dfs.columns = ['dm_average_returns_tcosts', 'coint_average_returns_tcosts', 'copula_average_returns_tcosts', 'dm_average_returns_no_tcosts', 'coint_average_returns_no_tcosts', 'copula_average_returns_no_tcosts']
all_dfs.to_excel('Data/all_average_returns.xlsx')